# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../output_data/cities.csv')
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,yulara,92,AU,1580416870,83,77.0,-25.24,130.99,4.70
1,1,marawi,40,PH,1580416871,89,72.0,8.00,124.29,5.01
2,2,pitimbu,20,BR,1580416871,94,73.4,-7.47,-34.81,16.11
3,3,ushuaia,75,AR,1580416772,81,48.2,-54.80,-68.30,33.33
4,4,mataura,60,NZ,1580416871,64,59.0,-46.19,168.86,21.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [11]:
# Store latitude and longitude in locations
locations = cities[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities["Humidity"].astype(float)

# Plot Heatmap
#https://jupyter-gmaps.readthedocs.io/en/latest/api.html
fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
city_list_good = cities.loc[(cities["Temp"] > 65) & (cities["Temp"] < 85) & (cities["Cloudiness"] == 0), :]
city_list_good = city_list_good.dropna(how='any')
city_list_good.reset_index(inplace=True)
del city_list_good['index']
city_list_good.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,7,cape town,0,ZA,1580416871,72,73.00,-33.93,18.42,9.17
1,21,kaeo,0,NZ,1580416873,87,75.99,-35.10,173.78,5.01
2,25,atuona,0,PF,1580416873,73,81.10,-9.80,-139.03,12.08
3,40,mar del plata,0,AR,1580416700,49,71.60,-38.00,-57.56,16.11
4,63,corowa,0,AU,1580416877,68,70.00,-36.03,146.38,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotels = []

for i in range(len(city_list_good)):
    lat = city_list_good.loc[i]['Lat']
    lng = city_list_good.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius" : 4000,
        "types" : "hotel",
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    json = requested.json()
    try:
        hotels.append(json['results'][0]['name'])
    except:
        hotels.append("")
city_list_good["Hotel Name"] = hotels
city_list_good = city_list_good.dropna(how='any')
city_list_good.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,7,cape town,0,ZA,1580416871,72,73.00,-33.93,18.42,9.17,Cape Town
1,21,kaeo,0,NZ,1580416873,87,75.99,-35.10,173.78,5.01,Kaeo
2,25,atuona,0,PF,1580416873,73,81.10,-9.80,-139.03,12.08,Atuona
3,40,mar del plata,0,AR,1580416700,49,71.60,-38.00,-57.56,16.11,Mar del Plata
4,63,corowa,0,AU,1580416877,68,70.00,-36.03,146.38,4.70,Rutherglen


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels_info = [info_box_template.format(**row) for index, row in city_list_good.iterrows()]
locations = city_list_good[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Display Map
markers

Markers(data_bounds=[(-59.92334126827008, -179.99999), (38.82684126827007, 179.99999)], markers=[Marker(locati…